# Tropical cyclone hazard for displacement risk modelling

This tutorial shows how to use the TC hazard event sets (a) and return periods (RP) maps (b).

Technical specifications.

Resolution: 150 arcsec 

**Emission scenarios considered:** SSP245, SSP370, SSP585.  
**GCMs considered:** .  
**(Future) years considered:** 1980-2018 (ERA-5) & 1995-2014 (20thcal GCM output) - 2041-2060, 2081-2100.  
**Return periods considered for flood maps:** 1, 10, 25, 50, 100, 250.   


In each folder, there is then a combination of RCP and year, and within each RCP_year folder, there is one tif file per return period for the latitude-longitude tile.

In [1]:
import os
import pandas as pd
import numpy as np

os.chdir('/Users/simonameiler/Documents/WCR/Displacement/global-displacement-risk') # change back to root folder, not "~/doc"
import coastal_flood_hazard, exposure, vulnerability

Select latitude-longitude tiles for country of interest only.

First, load exposure and get lat/lon max/mins from it.  
Then load the respective flood tiles.

## Load exposure from BEM

In [2]:
from climada.entity.exposures import Exposures

In [3]:
cntry_name = 'Sri Lanka'
reg = 'IO'

In [4]:
# Load the full dataframe, without further re-aggregation / processing other than adding centroids
gdf_bem_subcomps = exposure.gdf_from_bem_subcomps(cntry_name, opt='full')
gdf_bem_subcomps.head()

FileNotFoundError: [Errno 2] No such file or directory: '/cluster/work/climate/evelynm/IDMC_UNU/exposure/bem_cntry_files/lka_bem_1x1_valfis.csv'

In [ ]:
# filter and apply impf id
gdf_bem_subcomps = gdf_bem_subcomps[gdf_bem_subcomps.valhum>0.001] # filter out rows with basically no population
gdf_bem_subcomps["impf_TC"] = gdf_bem_subcomps.apply(lambda row: vulnerability.DICT_PAGER_TCIMPF_HAZUS[row.se_seismo], axis=1)

In [ ]:
# remove for now unnecessary cols and prepare gdf for CLIMADA Exposure
gdf_bem_subcomps.rename({'valhum' : 'value'}, axis=1)
for col in ['iso3', 'sector', 'valfis', 'se_seismo']:
    gdf_bem_subcomps.pop(col)

In [ ]:
gdf_bem_subcomps

In [ ]:
exp = Exposures(gdf_bem_subcomps)
exp.gdf.rename({'valhum': 'value'}, axis=1, inplace=True)
exp.value_unit = 'Pop. count'
exp.gdf['longitude'] = exp.gdf.geometry.x
exp.gdf['latitude'] = exp.gdf.geometry.y
exp.gdf = exp.gdf[~np.isnan(
    exp.gdf.latitude)]  # drop nan centroids
exp.gdf.head()

In [ ]:
print('Total population '+str(cntry_name)+': ' + "{:,.0f}".format(exp.gdf.value.sum()))

#### Get lat/lon min/max from exposure

In [ ]:
lat_min, lat_max, lon_min, lon_max = exp.gdf['latitude'].min(), exp.gdf['latitude'].max(), exp.gdf['longitude'].min(), exp.gdf['longitude'].max()

In [ ]:
lat_min, lat_max, lon_min, lon_max

## Load hazard

### a) Event set

In [ ]:
from climada.util.constants import SYSTEM_DIR
from climada.hazard import TropCyclone, Hazard
hazard_dir = SYSTEM_DIR/"hazard"/"present"

In [ ]:
tc_haz = TropCyclone.from_hdf5(hazard_dir.joinpath('TC_IO_0150as_MIT_H08.hdf5'))

In [ ]:
tc_haz.plot_intensity(event=0)

In [ ]:
tc_haz_sel = tc_haz.select(extent=(lon_min, lon_max, lat_min, lat_max))

### b) Hazard maps

In [ ]:
hazard_dir = SYSTEM_DIR/"hazard"/"RPmaps"

In [ ]:
haz_str= f"TC_{reg}_0150as_MIT_RP-maps.nc"

In [ ]:
file_path = hazard_dir.joinpath(haz_str)
print(file_path)

In [ ]:
import xarray as xr
from scipy import sparse
from climada.hazard.centroids.centr import Centroids

def read_netcdf_as_hazard(file_path):
    """
    Read NetCDF file containing exceedance intensity data for various return periods
    and convert it into a Hazard object.

    Args:
    - file_path (str): Path to the NetCDF file.

    Returns:
    - Hazard: Hazard object containing the data from the NetCDF file.
    """
    with xr.open_dataset(file_path) as ds:
        centroids = Centroids(lat=ds['latitude'].values, lon=ds['longitude'].values)
        
        # Create a list to hold intensity data arrays
        intensity_arrays = []
        return_periods = [1, 10, 25, 50, 100, 250]  # Specify your return periods
        for rp in return_periods:
            intensity_var = f'intensity_RP{rp}'
            intensity_arrays.append(ds[intensity_var].values)

        # Stack arrays vertically and convert to sparse matrix
        intensity_matrix = np.vstack(intensity_arrays)
        intensity_sparse = sparse.csr_matrix(intensity_matrix)

        # Create the Hazard object
        hazard = Hazard(
            haz_type='TC',
            units='m/s',
            centroids=centroids,
            event_id=np.arange(len(np.array(return_periods, dtype=int))),
            event_name=np.arange(len(np.array(return_periods, dtype=int))),
            date=np.arange(len(np.array(return_periods, dtype=int))),
            intensity=intensity_sparse,
            frequency=1 / np.array(return_periods)
        )

    return hazard

In [ ]:
RP_haz = read_netcdf_as_hazard(file_path)

In [ ]:
RP_haz.plot_intensity(event=0)

In [ ]:
RP_haz_sel = RP_haz.select(extent=(lon_min, lon_max, lat_min, lat_max))

In [ ]:
RP_haz_sel.plot_intensity(event=0)

## Impact functions

In [ ]:
impf_set_tc = vulnerability.IMPF_SET_TC_HAZUS

In [ ]:
from climada.entity import ImpactFunc, ImpactFuncSet
impf_set_tc_step = ImpactFuncSet()

In [ ]:
# The threshold of building damage after which all people are displaced. Below, no-one is displaced.
building_thresh = 0.55 # 55% iDMC v1; CIMA: 30% for Somalia to 60% for other countries. 

for imp_id in impf_set_tc.get_ids(haz_type='TC'):
    impf_set_tc.get_func(fun_id=imp_id)
    y = impf_set_tc.get_func(fun_id=imp_id)[0].intensity
    x = impf_set_tc.get_func(fun_id=imp_id)[0].mdd
    thresh = np.interp(building_thresh, x, y)
    print('ID: '+str(imp_id)+' - threshold stepfunction: '+str(thresh))
    impf_set_tc_step.append(
                ImpactFunc.from_step_impf(
                    intensity=(0,  thresh, thresh *10),
                    haz_type='TC',
                    impf_id=imp_id,
                    intensity_unit = 'm/s'
                )
    )

## Impacts

### Historical

##### a) event-based impact calculation

In [ ]:
from climada.engine import ImpactCalc

impcalc = ImpactCalc(exp, impf_set_tc_step, tc_haz_sel)
impact = impcalc.impact()

In [ ]:
print('Annual average displacement: ' + "{:,.0f}".format(impact.aai_agg))

In [ ]:
freq_curve = impact.calc_freq_curve()
freq_curve.plot()

In [ ]:
freq_curve = impact.calc_freq_curve(return_per=np.arange(1, 251, 1))
rp_indices = [0, 9, 24, 49, 99, 249]
pm_data = [freq_curve.impact[idx] for idx in rp_indices]
freq_curve.plot()

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

fig, ax = plt.subplots(figsize=(8, 6), subplot_kw={'projection': ccrs.PlateCarree()})

# Use the plot method and pass the GeoAxes
impact.plot_hexbin_eai_exposure(
    ignore_zero=True,
    pop_name=False,
    axis=ax
)

plt.show()

##### b) RP maps-based impact calculation

In [ ]:
impcalc_RP = ImpactCalc(exp, impf_set_tc_step, RP_haz_sel)
impact_RP = impcalc_RP.impact()

In [ ]:
print('Annual average displacement: ' + "{:,.0f}".format(impact_RP.aai_agg))

In [ ]:
freq_RP_curve = impact_RP.calc_freq_curve(return_per=1/RP_haz.frequency)
freq_RP_curve.plot()
pm_RP_data = [imp for imp in freq_RP_curve.impact]

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6), subplot_kw={'projection': ccrs.PlateCarree()})

impact_RP.plot_hexbin_eai_exposure(
    ignore_zero=True,
    pop_name=False,
    axis=ax
)

plt.show()

#### Compare results a) event-base vs. b) RP maps

In [ ]:
fq_dict = {'event-based': freq_curve,
           'RP maps': freq_RP_curve}

In [ ]:
import matplotlib.pyplot as plt

labels = list(fq_dict.keys())

fig, axis = plt.subplots()

for i, (plots, fq) in enumerate(fq_dict.items()):
    fq.plot(axis=axis, label=str(cntry_name)+' '+labels[i])

axis.legend(loc='center right', bbox_to_anchor=(1.45, 0.5))

In [ ]:
RPs = freq_RP_curve.return_per.tolist()

In [ ]:
# Create the AAD DataFrame
df_AAD = pd.DataFrame({
    "event-based": [impact.aai_agg],
    "RP maps": [impact_RP.aai_agg]
}, index=["AAD"])

# Create the PMD DataFrame with 'RP' as its index
df_PMD = pd.DataFrame({
    "event-based": pm_data,
    "RP maps": pm_RP_data
}, index=["RP_" + str(int(rp)) for rp in RPs])

# Concatenate df_AAD and df_PMD vertically
df_hist = pd.concat([df_AAD, df_PMD])

# Formatting the DataFrame
df_hist_display = df_hist.style.format({
    "event-based": "{:,.0f}",
    "RP maps": "{:,.0f}"
})

df_hist_display

In [ ]:
# Save the DataFrame to a CSV file
results_dir = SYSTEM_DIR/'results'
file_name = f'TC_disp_{cntry_name}_hist.csv'
df_hist.to_csv(results_dir.joinpath(file_name), index=False)

### Future

In [ ]:
reg = 'IO'
models = ['cesm2', 'cnrm6', 'ecearth6', 'fgoals', 'ipsl6', 'miroc6', 'mpi6', 'mri6', 'ukmo6']
scenario = ['20thcal', 'ssp245_2cal', 'ssp585_2cal']

In [ ]:
fut_haz_dict = {}
for gcm in models:
    for scen in scenario:
        haz_str_fut = f"TC_{reg}_0150as_MIT_{gcm}_{scen}_RP-maps.nc"
        file_path = hazard_dir.joinpath(haz_str_fut)
        print(file_path)
        haz = read_netcdf_as_hazard(file_path)
        haz_sel = haz.select(extent=(lon_min, lon_max, lat_min, lat_max))
        fut_haz_dict[gcm+'_'+scen] = haz_sel

In [ ]:
impact_dict = {}
for fut, haz in fut_haz_dict.items():
    impcalc_fut = ImpactCalc(exp, impf_set_tc_step, haz)
    impact_fut = impcalc_fut.impact()
    impact_dict[fut] = impact_fut

In [ ]:
aai_agg_dict = {}
pmd_dict = {}
for fut, imp in impact_dict.items():
    aai_agg_dict[fut] = imp.aai_agg
    freq_curve = imp.calc_freq_curve(return_per=1/RP_haz.frequency)
    pm_data = [fq_imp for fq_imp in freq_curve.impact]
    pmd_dict[fut] = pm_data

In [ ]:
from collections import defaultdict

# Initialize a dictionary to store aggregated values by scenario
scenario_data = defaultdict(list)

# Process each item in the dictionary
for key, value in aai_agg_dict.items():
    parts = key.rsplit('_', 2)
    model = parts[0]
    scenario = parts[1]
    scenario_data[scenario].append(value)

# Calculate the mean for each scenario
scenario_means = {scenario: np.mean(values) for scenario, values in scenario_data.items()}

scenario_means

In [ ]:
# Create DataFrame
data = {
    'Model': [],
    'Scenario': [],
    'Period': [],
    'AAD': []
}

for key, value in aai_agg_dict.items():
    parts = key.split('_')
    if len(parts) == 3: 
        model, scenario, period = parts
    elif len(parts) == 2: 
        model, scenario = parts
        period = "hist"
    else:
        continue

    data['Model'].append(model)
    data['Scenario'].append(scenario)
    data['Period'].append(period)
    data['AAD'].append(value)

df = pd.DataFrame(data)

# Iterate and add new columns for each RP value
for rp, rp_value in zip([1, 10, 25, 50, 100, 250], zip(*pmd_dict.values())):
    df[f'RP_{rp}'] = rp_value

df

In [ ]:
# Save the DataFrame to a CSV file
results_dir = SYSTEM_DIR/'results'
file_name = f'TC_disp_{cntry_name}_RP-maps.csv'
df.to_csv(results_dir.joinpath(file_name), index=False)